In [5]:
import numpy as np

#Read data

X = []
samples_x = []

f = open("genotypes.csv","r")

for i,line in enumerate(f):
    if(i == 0):
       continue
    
    sv = line.strip().split(",")
    samples_x.append(sv[0])
    
    row = []
    for element in sv[1:]:
        row.append(element)
    X.append(row)
    
f.close()

X = np.array(X)
samples_x = np.array(samples_x)

# Read phenotype values

y = []
samples_y = []

f = open("phenotype_values.csv","r")

for i,line in enumerate(f):
    if(i == 0):
       continue
    sv = line.strip().split(",")
    samples_y.append(sv[0])
    y.append(float(sv[1]))
    
f.close()

y = np.array(y)
samples_y = np.array(samples_y)

In [6]:
#Replace missing values with np.nan

X = np.where(X == "?", np.nan, X)

#Encode nucleotide values

for j in range (X.shape[1]):
    
    values, counts = np.unique(X[:,j], return_counts=True)
    values=values[0:2]
    counts=counts[0:2]

    X[:,j] = np.where(X[:,j] == values[np.argmax(counts)], 0.0, X[:,j])
    X[:,j] = np.where(X[:,j] == values[np.argmin(counts)], 2.0, X[:,j])

#Convert array from string to float
X = X.astype(np.float_)

[['G' 'G' 'G' ... 'nan' 'T' 'A']
 ['G' 'G' 'G' ... 'A' 'T' 'A']
 ['A' 'G' 'A' ... 'A' 'nan' 'A']
 ...
 ['G' 'A' 'G' ... 'T' 'T' 'A']
 ['G' 'G' 'A' ... 'A' 'T' 'A']
 ['G' 'G' 'G' ... 'A' 'T' 'C']]


In [7]:
# Assess missing values per sample

sample_indices_to_remove = []
for i in range(X.shape[0]):
    missing = np.isnan(X[i,:]).sum()/X.shape[1]*100
    if missing>50:
        print("Sample %d has %.2f missing values" % (i,missing))
        sample_indices_to_remove.append(i)

if (len(sample_indices_to_remove) == 0):
    print("All samples have less than 50 % missing values")

All samples have less than 50 % missing values


In [8]:
# Impute missing values on training data

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="most_frequent")

X = imputer.fit_transform(X)

In [9]:
# Match genotypes with phenotype values
truth_table = (samples_x[:,np.newaxis]==samples_y)
ind = np.where(truth_table==True)

samples_x = samples_x[ind[0]]
samples_y = samples_y[ind[1]]

X = X[ind[0],:]
y = y[ind[1]]

In [10]:
#standardise the data
from sklearn.preprocessing import StandardScaler 

scaler = StandardScaler()
Xs = scaler.fit_transform(X)

#compute the covariance matrix on the standardised data
n = X.shape[0]
C = 1/(n-1) * np.dot(Xs.T,Xs)

#eigendecompose the matrix  𝐂  into its eigenvalues and eigenvector
import numpy.linalg as linalg
d, V = linalg.eig(C)

#sort the eigenvalues in decreasing order, re-sort the columns of the eigenvector matrix 𝑉  using the indices from the sorting of the eigenvalues
ind = np.argsort(d)[::-1]
d = d[ind]
V = V[:,ind]

Xr = np.dot(Xs,V[:,0:1000])

#calculate how much of the total variance the PCs account for
#ratios_variance_explained = d/d.sum()
#va = ratios_variance_explained[0:1000].sum()
#print("First 1000 PCs account for %.2f %% of the total variance!" % (va*100))

[[-8.68376349e+00+0.j -5.60131328e+00+0.j  9.34610142e+00+0.j ...
   7.62645725e-01+0.j -3.91271658e-01+0.j  4.47480771e-01+0.j]
 [-9.09791484e+00+0.j  2.01712253e+00+0.j  5.23998996e+00+0.j ...
   8.30489368e-01+0.j  1.62805750e+00+0.j  9.43439445e-02+0.j]
 [ 4.01402629e+01+0.j -1.81938059e+01+0.j -7.44733592e+00+0.j ...
   1.54980833e+00+0.j -1.75373738e-02+0.j  3.79840819e-01+0.j]
 ...
 [ 3.36663967e+01+0.j  6.91548467e+00+0.j  2.93889646e+00+0.j ...
   1.65162263e+00+0.j -2.26837294e+00+0.j -5.47490890e-01+0.j]
 [-7.26679778e+00+0.j  3.66994226e+00+0.j  5.88215878e+00+0.j ...
  -1.91928172e+00+0.j -1.31333746e+00+0.j -1.05577672e+00+0.j]
 [-9.36925283e+00+0.j  9.02793145e+00+0.j -3.59236321e+00+0.j ...
   1.41718784e-01+0.j  1.16382330e+00+0.j  1.65280364e-01+0.j]]
First 1000 PCs account for 90.89 % of the total variance!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: ComplexWarning: Casting complex values to real discards the imaginary part


In [11]:
# Split into train data and test data

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Xr,y,test_size=0.2,random_state=73)

X_train = X_train.astype(np.float_)
X_test = X_test.astype(np.float_)
y_train = y_train.astype(np.float_)
y_test = y_test.astype(np.float_)

#print("Full Data:\t" + str(Xr.shape))
#print("Train Data:\t" + str(X_train.shape))
#print("Test Data:\t" + str(X_test.shape))

Full Data:	(1826, 1000)
Train Data:	(1460, 1000)
Test Data:	(366, 1000)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [ ]:
#Example Support Vector Regression

from sklearn.svm import SVR
import sklearn.metrics as metrics

Cs = np.arange(1,251,10)
Gamma = ["auto","scale"]
Epsilon = np.arange(0.1,7.1,0.1)

r2_train = []
r2_test = []

for c in Cs:
    for g in Gamma:
        for e in Epsilon:
            svr_poly = SVR(C=c, gamma=g, epsilon=e)
            svr_poly.fit(X_train, y_train) #training step, just stores data
            svr_predictions_training = svr.predict(X_train)
            svr_predictions_testing = svr.predict(X_test)
            r2_training = metrics.r2_score(y_train, svr_predictions_training)
            r2_testing = metrics.r2_score(y_test, svr_predictions_testing)
            r2_train.append(r2_training)
            r2_test.append(r2_testing)
    
best_c = Cs[np.argmax(r2_test)]
best_g = Gamma[np.argmax(r2_test)]
best_e = Epsilon[np.argmax(r2_test)]
print("Best_c = " + str(best_c))
print("Best_g = " + str(best_g))
print("Best_e = " + str(best_e))
print("Best R2: %.4f" %np.max(r2_test))